<a href="https://colab.research.google.com/github/martin-mirantes/MCP/blob/main/notebooks/search/00-quick-start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic search quick start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/00-quick-start.ipynb)

This interactive notebook will introduce you to some basic operations with Elasticsearch, using the official [Elasticsearch Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html).
You'll perform semantic search using [Sentence Transformers](https://www.sbert.net) for text embedding. Learn how to integrate traditional text-based search with semantic search, for a hybrid search system.

## Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?onboarding_token=vectorsearch&utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

Once logged in to your Elastic Cloud account, go to the [Create deployment](https://cloud.elastic.co/deployments/create) page and select **Create deployment**. Leave all settings with their default values.

## Install packages and import modules

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to install the `elasticsearch` Python client.

In [ ]:
!pip install -qU "elasticsearch<9" #sentence-transformers==2.7.0

# Setup the Embedding Model

For this example, we're using `all-MiniLM-L6-v2`, part of the `sentence_transformers` library. You can read more about this model on [Huggingface](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

## Initialize the Elasticsearch client

Now we can instantiate the [Elasticsearch python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html), providing the cloud id and password in your deployment.

In [ ]:
from elasticsearch import Elasticsearch
from getpass import getpass

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# Create the client instance
client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

If you're running Elasticsearch locally or self-managed, you can pass in the Elasticsearch host instead. [Read more](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#_verifying_https_with_certificate_fingerprints_python_3_10_or_later) on how to connect to Elasticsearch locally.

### Enable Telemetry

Knowing that you are using this notebook helps us decide where to invest our efforts to improve our products. We would like to ask you that you run the following code to let us gather anonymous usage statistics. See [telemetry.py](https://github.com/elastic/elasticsearch-labs/blob/main/telemetry/telemetry.py) for details. Thank you!

In [ ]:
!curl -O -s https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/telemetry/telemetry.py
from telemetry import enable_telemetry

client = enable_telemetry(client, "00-quick-start")

### Test the Client
Before you continue, confirm that the client has connected with this test.

In [ ]:
print(client.info())

## Index some test data

Our client is set up and connected to our Elastic deployment.
Now we need some data to test out the basics of Elasticsearch queries.
We'll use a small index of books with the following fields:

- `title`
- `authors`
- `publish_date`
- `num_reviews`
- `publisher`

### Create an index

First ensure that you do not have a previously created index with the name `book_index`.

In [ ]:
client.indices.delete(index="book_index", ignore_unavailable=True)

🔐 NOTE: at any time you can come back to this section and run the `delete` function above to remove your index and start from scratch.

Let's create an Elasticsearch index with the correct mappings for our test data.

In [ ]:
# Define the mapping
mappings = {
    "properties": {
        "title_vector": {
            "type": "dense_vector",
            "dims": 384,
            "index": "true",
            "similarity": "cosine",
        }
    }
}

# Create the index
client.indices.create(index="book_index", mappings=mappings)

### Index test data

Run the following command to upload some test data, containing information about 10 popular programming books from this [dataset](https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json).
`model.encode` will encode the text into a vector on the fly, using the model we initialized earlier.

In [ ]:
import json
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json"
response = urlopen(url)
books = json.loads(response.read())

operations = []
for book in books:
    operations.append({"index": {"_index": "book_index"}})
    # Transforming the title into an embedding using the model
    book["title_vector"] = model.encode(book["title"]).tolist()
    operations.append(book)
client.bulk(index="book_index", operations=operations, refresh=True)

## Aside: Pretty printing Elasticsearch responses

Your API calls will return hard-to-read nested JSON.
We'll create a little function called `pretty_response` to return nice, human-readable outputs from our examples.

In [ ]:
def pretty_response(response):
    if len(response["hits"]["hits"]) == 0:
        print("Your search returned no results.")
    else:
        for hit in response["hits"]["hits"]:
            id = hit["_id"]
            publication_date = hit["_source"]["publish_date"]
            score = hit["_score"]
            title = hit["_source"]["title"]
            summary = hit["_source"]["summary"]
            publisher = hit["_source"]["publisher"]
            num_reviews = hit["_source"]["num_reviews"]
            authors = hit["_source"]["authors"]
            pretty_output = f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nPublisher: {publisher}\nReviews: {num_reviews}\nAuthors: {authors}\nScore: {score}"
            print(pretty_output)

## Making queries

Now that we have indexed the books, we want to perform a semantic search for books that are similar to a given query.
We embed the query and perform a search.

In [ ]:
response = client.search(
    index="book_index",
    knn={
        "field": "title_vector",
        "query_vector": model.encode("clear"),
        "k": 3,
        "num_candidates": 3,
    },
)

pretty_response(response)

## Filtering

Filter context is mostly used for filtering structured data. For example, use filter context to answer questions like:

- _Does this timestamp fall into the range 2015 to 2016?_
- _Is the status field set to "published"?_

Filter context is in effect whenever a query clause is passed to a filter parameter, such as the `filter` or `must_not` parameters in a `bool` query.

[Learn more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-filter-context.html#filter-context) about filter context in the Elasticsearch docs.

### Example: Keyword Filtering

This is an example of adding a keyword filter to the query.

The example retrieves the top books that are similar to "javascript books" based on their title vectors, and also Addison-Wesley as publisher.

In [ ]:
response = client.search(
    index="book_index",
    knn={
        "field": "title_vector",
        "query_vector": model.encode("javascript books"),
        "k": 10,
        "num_candidates": 100,
        "filter": {"term": {"publisher.keyword": "addison-wesley"}},
    },
)

pretty_response(response)

In [ ]:
from datetime import datetime, timedelta
# Calcula la fecha de hace dos años desde hoy
two_years_ago = (datetime.now() - timedelta(days=8*365)).strftime('%Y-%m-%d')

response = client.search(
    index="book_index",
    knn={
        "field": "title_vector",
        "query_vector": model.encode("javascript books"), # o la consulta que desees
        "k": 10,
        "num_candidates": 100,
        "filter": {
            "range": {
                "publish_date": {
                    "gte": two_years_ago,
                    "lte": "now" # Puedes usar "now" para la fecha actual
                }
            }
        },
    },
)

# Asume que tienes una función pretty_response definida
# Ejemplo:
# import json
# def pretty_response(response):
#     print(json.dumps(response.body, indent=2, ensure_ascii=False))

pretty_response(response)

1. Buscar Libros por Autor ✍️
Puedes buscar utilizando el campo authors (analizado, para búsquedas flexibles) o authors.keyword (no analizado, para búsquedas exactas).

Búsqueda flexible (usando authors de tipo text)
Esto encontrará libros donde "Gabriel García" aparezca en el campo de autores, incluso si hay otros autores o si el nombre tiene variaciones que el analizador pueda manejar.

Python

In [ ]:
import json
def pretty_print(response):
    """Función auxiliar para imprimir respuestas JSON de forma legible."""
    print(json.dumps(response, indent=2, ensure_ascii=False))

In [ ]:
# Nombre del índice
INDEX_NAME = "book_index"

print("## 1.1 Búsqueda flexible de autor:")
response = client.search(
    index=INDEX_NAME,
    query={
        "match": {
            "authors": "ericc mathes"
        }
    },
    size=5 # Limitar a 5 resultados
)
pretty_response(response.body)

In [ ]:
print("\n## Búsqueda flexible de autor (incluyendo solo 'title', 'authors', 'publish_date'):")
response_specific_fields = client.search(
    index=INDEX_NAME,
    query={
        "match": {
            "authors": "eric"
        }
    },
    size=1,
    _source=["title", "authors", "publish_date"] # <--- AQUÍ SE ESPECIFICAN LOS CAMPOS A INCLUIR
)
pretty_print(response_specific_fields.body)

In [ ]:
print("## Búsqueda flexible de autor (excluyendo 'title_vector'):")
response = client.search(
    index=INDEX_NAME,
    query={
        "match": {
            "authors": "eric"
        }
    },
    size=1, # Limitar a 1 resultado para el ejemplo
    _source_excludes=["title_vector"] # <--- AQUÍ SE EXCLUYE EL CAMPO
)
pretty_print(response.body)

In [ ]:
print("## Búsqueda por Prefijo (ej: 'eri'):")
query_term = "eric"
response = client.search(
    index=INDEX_NAME,
    query={
        "match_phrase_prefix": {
            "authors": {
                "query": query_term
            }
        }
    },
    size=5,
    _source_excludes=["title_vector"]
)
pretty_print(response.body)
# Debería encontrar 'eric matthes'

In [ ]:
print("\n## Búsqueda con Fuzziness (ej: 'erric'):")
query_term = "rric"
response = client.search(
    index=INDEX_NAME,
    query={
        "match": {
            "authors": {
                "query": query_term,
                "fuzziness": "AUTO" # O un valor como 1 o 2 (distancia de Levenshtein)
                                # "AUTO" genera distancias de edición basadas en la longitud del término.
            }
        }
    },
    size=5,
    _source_excludes=["title_vector"]
)
pretty_print(response.body)
# Debería encontrar 'eric matthes' si la fuzziness es adecuada.

In [ ]:
print("\n## Búsqueda con Comodines usando query_string:")

# Ejemplo para "eri" (como prefijo)
query_wildcard_prefix = "eri*"
print(f"--- Buscando con: {query_wildcard_prefix} ---")
response_prefix = client.search(
    index=INDEX_NAME,
    query={
        "query_string": {
            "query": query_wildcard_prefix,
            "default_field": "authors"
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_prefix.body)



In [ ]:
# Ejemplo para "ric" (como parte del término)
query_wildcard_contains = "*ric*" # Podría ser "*ric" si solo buscas al final
print(f"\n--- Buscando con: {query_wildcard_contains} ---")
response_contains = client.search(
    index=INDEX_NAME,
    query={
        "query_string": {
            "query": query_wildcard_contains,
            "default_field": "authors"
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_contains.body)
# Esta consulta ('*ric*') debería encontrar 'eric matthes'.

In [ ]:
print("\n## Búsqueda con simple_query_string:")

# Para prefijo "eri"
query_sqs_prefix = "eri*"
print(f"--- Buscando con simple_query_string (prefijo): {query_sqs_prefix} ---")
response_sqs_prefix = client.search(
    index=INDEX_NAME,
    query={
        "simple_query_string": {
            "query": query_sqs_prefix,
            "fields": ["authors"], # Especifica los campos donde buscar
            "default_operator": "AND"
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_sqs_prefix.body)


# Para typo "erric" (usando fuzziness)
query_sqs_fuzzy = "erric~1" # El ~1 indica una distancia de edición de 1
print(f"\n--- Buscando con simple_query_string (fuzziness): {query_sqs_fuzzy} ---")
response_sqs_fuzzy = client.search(
    index=INDEX_NAME,
    query={
        "simple_query_string": {
            "query": query_sqs_fuzzy,
            "fields": ["authors"],
            "default_operator": "AND"
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_sqs_fuzzy.body)


# Para "ric" como parte de "eric"
query_sqs_contains = "*ric*"
print(f"\n--- Buscando con simple_query_string (contiene): {query_sqs_contains} ---")
response_sqs_contains = client.search(
    index=INDEX_NAME,
    query={
        "simple_query_string": {
            "query": query_sqs_contains,
            "fields": ["authors"],
            "default_operator": "AND"
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_sqs_contains.body)

In [ ]:
print("\n## Probando con query_string para '*ric*':")
query_term = "*ric*"
response_qs = client.search(
    index=INDEX_NAME,
    query={
        "query_string": {
            "query": query_term,
            "fields": ["authors"], # Puedes usar 'fields' o 'default_field'
            # "analyze_wildcard": False # Es el valor por defecto, pero puedes ser explícito
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_qs.body)

In [ ]:
print("\n## Probando simple_query_string simplificada para '*ric*':")
query_term = "*ric*"
response_sqs_simple = client.search(
    index=INDEX_NAME,
    query={
        "simple_query_string": {
            "query": query_term,
            "fields": ["authors"]
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_sqs_simple.body)

In [ ]:
GET book_index/_termvectors/_NVoQJcBYS182ubtUI05
{
  "fields": ["authors"]
}

In [ ]:
print("\n## 1.2 Búsqueda exacta de autor:")
response = client.search(
    index=INDEX_NAME,
    query={
        "term": {
            "authors.keyword": "david"
        }
    },
    size=5
)
pretty_response(response.body)

In [ ]:
print("\n## 2. Agregación de autores únicos:")
response = client.search(
    index=INDEX_NAME,
    size=0,  # No necesitamos los documentos, solo la agregación
    aggs={
        "unique_authors": {
            "terms": {
                "field": "authors.keyword",
                "size": 20  # Número de autores únicos a mostrar
            }
        }
    }
)
pretty_print(response.body['aggregations'])

In [ ]:
print("\n## 3. Destacar términos en autores:")
response = client.search(
    index=INDEX_NAME,
    query={
        "match": {
            "authors": "Martin" # Buscar libros que tengan "Martin" en los autores
        }
    },
    highlight={
        "fields": {
            "authors": {} # Configuración de resaltado por defecto para el campo 'authors'
        }
    },
    size=3,
    _source_excludes=["title_vector"]
)
pretty_print(response.body)

In [ ]:
print("\n## 4. Libros con más de un autor:")
response = client.search(
    index=INDEX_NAME,
    query={
        "bool": {
            "filter": {
                "script": {
                    "script": {
                        "source": "doc['authors.keyword'].size() > params.count",
                        "params": {
                            "count": 1
                        }
                    }
                }
            }
        }
    },
    size=5,
    _source=["title", "authors"] # Solo traer título y autores
)
pretty_print(response.body)

In [ ]:
print("\n## 5. Libros con más de dos autores:")
response = client.search(
    index=INDEX_NAME,
    query={
        "bool": {
            "filter": {
                "script": {
                    "script": {
                        "source": "doc['authors.keyword'].size() > params.count",
                        "params": {
                            "count": 2
                        }
                    }
                }
            }
        }
    },
    size=5,
    _source=["title", "authors"]
)
pretty_print(response.body)

In [ ]:
print("\n## 6. Facet por cantidad de autores:")
response = client.search(
    index=INDEX_NAME,
    size=0, # No necesitamos los documentos, solo la agregación
    aggs={
        "authors_count_facet": {
            "terms": {
                "script": {
                    "source": "doc['authors.keyword'].size()", # Devuelve el número de autores
                    "lang": "painless"
                },
                "size": 10 # Muestra los 10 conteos de autores más comunes
            }
        }
    }
)
pretty_print(response.body['aggregations'])

In [ ]:
from elasticsearch import Elasticsearch

# Asume que 'client' ya está inicializado
# client = Elasticsearch(...)
# INDEX_NAME = "book_index" # O un nuevo nombre como "book_index_ngram"

# 1. Definir settings y mappings
index_settings = {
    "analysis": {
        "analyzer": {
            "custom_trigram_analyzer": {
                "tokenizer": "standard",
                "filter": [
                    "lowercase",
                    "trigram_token_filter"
                ]
            }
        },
        "filter": {
            "trigram_token_filter": {
                "type": "ngram",
                "min_gram": 3,
                "max_gram": 3
            }
        }
    }
}

index_mappings = {
    "properties": {
        "authors": {
            "type": "text",
            "analyzer": "custom_trigram_analyzer",
            "fields": {
                "keyword": {
                    "type": "keyword",
                    "ignore_above": 256
                }
            }
        },
        "num_reviews": {"type": "long"},
        "publish_date": {"type": "date"},
        "publisher": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}
        },
        "summary": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}
        },
        "title": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}
        },
        "title_vector": {
            "type": "dense_vector",
            "dims": 384,
            "index": True,
            "similarity": "cosine",
            "index_options": {"type": "int8_hnsw", "m": 16, "ef_construction": 100}
        }
    }
}

# (Opcional) Nombre del nuevo índice, o puedes usar el mismo si lo vas a borrar primero
NEW_INDEX_NAME = "book_index_trigram" # O usa tu INDEX_NAME original

# 2. (Opcional) Borrar el índice antiguo si vas a reutilizar el nombre y estás en desarrollo
if client.indices.exists(index=NEW_INDEX_NAME):
    print(f"Borrando índice existente: {NEW_INDEX_NAME}")
    client.indices.delete(index=NEW_INDEX_NAME)

# 3. Crear el nuevo índice con la configuración y mapeo
print(f"Creando índice: {NEW_INDEX_NAME}")
client.indices.create(
    index=NEW_INDEX_NAME,
    settings=index_settings,
    mappings=index_mappings
)
print("Índice creado con éxito.")

# 4. Re-poblar tus datos
#    Esto dependerá de cómo cargas tus datos originalmente.
#    Por ejemplo, si tienes una lista de documentos:
#    docs = [
#        {"authors": ["eric matthes"], "title": "Python Crash Course", ...},
#        ...
#    ]
#    for i, doc in enumerate(docs):
#        client.index(index=NEW_INDEX_NAME, id=i, document=doc)
#    print(f"{len(docs)} documentos indexados.")

# Si tienes un índice existente (ej. "book_index") y quieres mover los datos:
print("Iniciando reindexación...")
client.reindex(
     body={
         "source": {"index": "book_index"}, # Tu índice original
         "dest": {"index": NEW_INDEX_NAME}
     },
     request_timeout=300 # Aumenta el timeout si es necesario
 )
print("Reindexación completada.")

In [ ]:
# Asume que 'client' está configurado y NEW_INDEX_NAME es el índice con N-grams
# (y que has re-poblado los datos en este nuevo índice)

# Ejemplo: Buscar "ric"
search_term = "erric"

print(f"\nBuscando '{search_term}' en el campo 'authors' (N-gram):")
response = client.search(
    index=NEW_INDEX_NAME, # Asegúrate de consultar el índice correcto
    query={
        "match": {
            "authors": search_term
        }
    },
    size=5,
    _source=["title", "authors"]
)
# from your_previous_code import pretty_print # si la tienes definida
# pretty_print(response.body)
print(json.dumps(response.body, indent=2, ensure_ascii=False))

# Esto debería encontrar "eric matthes" porque "eric" genera el trigrama "ric",
# y el término de búsqueda "ric" (al ser de longitud 3) se convierte en el token "ric".

# [01-keyword-querying-filtering.ipynb](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/01-keyword-querying-filtering.ipynb#scrollTo=wMbAEseBzD5S)
## Querying
🔐 NOTE: to run the queries that follow you need the `book_index` dataset from our [quick start](https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/search/00-quick-start.ipynb). If you haven't worked through the quick start, please follow the steps described there to create an Elasticsearch deployment with the dataset in it, and then come back to run the queries here.

In the query context, a query clause answers the question _“How well does this document match this query clause?”_. In addition to deciding whether or not the document matches, the query clause also calculates a relevance score in the `_score `metadata field.

### Full text queries

Full text queries enable you to search analyzed text fields such as the body of an email. The query string is processed using the same analyzer that was applied to the field during indexing.

* **match**.
    The standard query for performing full text queries, including fuzzy matching and phrase or proximity queries.
* **multi-match**.
    The multi-field version of the match query.

##Match query

In [ ]:
response = client.search(
    index="book_index", query={"match": {"summary": {"query": "program"}}}
)

pretty_response(response)

##Multi-Match query

In [ ]:
response = client.search(
    index="book_index",
    query={"multi_match": {"query": "programming", "fields": ["summary", "title"]}},
)

pretty_response(response)

Individual fields can be boosted with the caret (^) notation. Note in the following query how the score of the results that have "JavaScript" in their title is multiplied.

In [ ]:
response = client.search(
    index="book_index",
    query={"multi_match": {"query": "javascript", "fields": ["summary", "title^3"]}},
)

pretty_response(response)

## Term-level Queries
You can use term-level queries to find documents based on precise values in structured data. Examples of structured data include date ranges, IP addresses, prices, or product IDs.

### Term search
Returns document that contain exactly the search term.

In [ ]:
response = client.search(
    index="book_index", query={"term": {"publisher.keyword": "addison-wesley"}}
)

pretty_response(response)

In [ ]:
response = client.search(
    index="book_index", query={"term": {"authors.keyword": "david thomas"}}
)

pretty_response(response)

### Range search
Returns documents that contain terms within a provided range.

The following example returns books that have at least 45 reviews.

In [ ]:
response = client.search(
    index="book_index", query={"range": {"num_reviews": {"gte": 45}}}
)

pretty_response(response)

### Prefix search

Returns documents that contain a specific prefix in a provided field.

[Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-prefix-query.html)

In [ ]:
response = client.search(
    index="book_index", query={"prefix": {"title": {"value": "jav"}}}
)

pretty_response(response)

### Fuzzy search

Returns documents that contain terms similar to the search term, as measured by a Levenshtein edit distance.

An edit distance is the number of one-character changes needed to turn one term into another. These changes can include:

* Changing a character (box → fox)
* Removing a character (black → lack)
* Inserting a character (sic → sick)
* Transposing two adjacent characters (act → cat)

[Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-fuzzy-query.html)

In [ ]:
response = client.search(
    index="book_index", query={"fuzzy": {"authors": {"value": "ric"}}}
)

pretty_response(response)

## Combining Query Conditions

Compound queries wrap other compound or leaf queries, either to combine their results and scores, or to change their behaviour. They also allow you to switch from query to filter context, but that will be covered later in the Filtering section.

#### bool.must (AND)
The clauses must appear in matching documents and will contribute to the score. This effectively performs an "AND" logical operation on the given sub-queries.

In [ ]:
response = client.search(
    index="book_index",
    query={
        "bool": {
            "must": [
                {"term": {"publisher.keyword": "addison-wesley"}},
                {"term": {"authors.keyword": "richard helm"}},
            ]
        }
    },
)

pretty_response(response)

#### bool.should (OR)

The clause should appear in the matching document. This performs an "OR" logical operation on the given sub-queries.

In [ ]:
response = client.search(
    index="book_index",
    query={
        "bool": {
            "should": [
                {"term": {"publisher.keyword": "addison-wesley"}},
                {"term": {"authors.keyword": "douglas crockford"}},
            ]
        }
    },
)

pretty_response(response)

## Filtering

In a filter context, a query clause answers the question *“Does this document match this query clause?”* The answer is a simple Yes or No — no scores are calculated. Filter context is mostly used for filtering structured data, for example:
* Does this `timestamp` fall into the range 2015 to 2016?
* Is the `status` field set to `"published"`?

Filter context is in effect whenever a query clause is passed to a `filter` parameter, such as the `filter` or `must_not` parameters in the `bool` query.

[Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html)

### bool.filter

The clause (query) must appear for the document to be included in the results. Unlike query context searches such as `term`, `bool.must` or `bool.should`, a matching `score` isn't calculated because filter clauses are executed in filter context.

In [ ]:
response = client.search(
    index="book_index",
    query={"bool": {"filter": [{"term": {"publisher.keyword": "prentice hall"}}]}},
)

pretty_response(response)

### bool.must_not
The clause (query) must not appear in the matching documents. Because this query also runs in filter context, no scores are calculated; the filter just determines if a document is included in the results or not.

In [ ]:
response = client.search(
    index="book_index",
    query={"bool": {"must_not": [{"range": {"num_reviews": {"lte": 45}}}]}},
)

pretty_response(response)

### Using Filters with Queries
Filters are often added to search queries with the intention of limiting the search to a subset of the documents. A filter can cleanly eliminate documents from a search, without altering the relevance scores of the results.

The next example returns books that have the word "javascript" in their title, only among the books that have more than 45 reviews.

In [ ]:
response = client.search(
    index="book_index",
    query={
        "bool": {
            "must": [{"match": {"title": {"query": "javascript"}}}],
            "must_not": [{"range": {"num_reviews": {"lte": 45}}}],
        }
    },
)

pretty_response(response)

# 02 Hybrid Search using RRF

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/02-hybrid-search.ipynb)

In this example we'll use the reciprocal rank fusion algorithm to combine the results of BM25 and kNN semantic search.
We'll use the same dataset we used in our [quickstart](https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/search/00-quick-start.ipynb) guide.

You can use RRF for hybrid search out of the box, without any additional configuration. This example demonstrates how RRF ranking works at a basic level.

# Install packages and initialize the Elasticsearch Python client

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to `pip` install the packages we need for this example.

In [ ]:
!pip install -qU "elasticsearch<9" sentence-transformers==2.7.0

Next we need to import the `elasticsearch` module and the `getpass` module.
`getpass` is part of the Python standard library and is used to securely prompt for credentials.

In [ ]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from getpass import getpass

model = SentenceTransformer("all-MiniLM-L6-v2")

Now we can instantiate the Python Elasticsearch client.
First we prompt the user for their password and Cloud ID.

🔐 NOTE: `getpass` enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [ ]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# Create the client instance
client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

### Enable Telemetry

Knowing that you are using this notebook helps us decide where to invest our efforts to improve our products. We would like to ask you that you run the following code to let us gather anonymous usage statistics. See [telemetry.py](https://github.com/elastic/elasticsearch-labs/blob/main/telemetry/telemetry.py) for details. Thank you!

In [ ]:
!curl -O -s https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/telemetry/telemetry.py
from telemetry import enable_telemetry

client = enable_telemetry(client, "02-hybrid-search")

### Test the Client
Before you continue, confirm that the client has connected with this test.

In [ ]:
print(client.info())

Refer to https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect to a self-managed deployment.

Read https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect using API keys.


## Pretty printing Elasticsearch responses

Let's add a helper function to print Elasticsearch responses in a readable format. This function is similar to the one that was used in the [quickstart](https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/search/00-quick-start.ipynb) guide.

In [ ]:
def pretty_response(response):
    if len(response["hits"]["hits"]) == 0:
        print("Your search returned no results.")
    else:
        for idx, hit in enumerate(response["hits"]["hits"], start=1):
            id = hit["_id"]
            publication_date = hit["_source"]["publish_date"]
            score = hit["_score"]
            title = hit["_source"]["title"]
            summary = hit["_source"]["summary"]
            pretty_output = f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nRank: {idx}\nScore: {score}"
            print(pretty_output)

# Querying Documents with Hybrid Search

🔐 NOTE: Before you can run the query in this section, you need the `book_index` dataset from our [quick start](https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/search/00-quick-start.ipynb). If you haven't worked through the quick start, please follow the steps described there to create an Elasticsearch deployment with the dataset in it, and then come back to run the query here.

Now we need to perform a query using two different search strategies:
- Semantic search using the "all-MiniLM-L6-v2" embedding model
- Keyword search using the "title" field

We then use [Reciprocal Rank Fusion (RRF)](https://www.elastic.co/guide/en/elasticsearch/reference/current/rrf.html) to balance the scores to provide a final list of documents, ranked in order of relevance. RRF is a ranking algorithm for combining results from different information retrieval strategies.

Note: With the retriever API, _score contains the document’s relevance score, and the rank is simply the position in the results (first result is rank 1, etc.).

In [ ]:
response = client.search(
    index="book_index",
    size=3,
    retriever={
        "rrf": {
            "retrievers": [
                {"standard": {"query": {"match": {"summary": "python programming"}}}},
                {
                    "knn": {
                        "field": "title_vector",
                        "query_vector": model.encode("python programming").tolist(),
                        "k": 3,
                        "num_candidates": 3,
                    }
                },
            ]
        }
    },
    source_excludes=["title_vector"] # <--- AQUÍ SE EXCLUYE EL CAMPO
)
pretty_response(response)
pretty_print(response.body)

# 03 Semantic Search using ELSER v2 text expansion

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/03-ELSER.ipynb)


Learn how to use the [ELSER](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-elser.html) for text expansion-powered semantic search.

**`Note:`** This notebook demonstrates how to use ELSER model `.elser_model_2` model which offers an improved retrieval accuracy.

If you have set up an index with ELSER model `.elser_model_1`, and would like to upgrade to ELSER v2 model - `.elser_model_2`, Please follow instructions from the notebook on [how to upgrade an index to use elser model](../model-upgrades/upgrading-index-to-use-elser.ipynb)

# Install and Connect

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to `pip` install the following packages:

- `elasticsearch`


In [ ]:
!pip install -qU "elasticsearch<9"

Next, we need to import the modules we need.
🔐 NOTE: `getpass` enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

In [ ]:
from elasticsearch import Elasticsearch, helpers, exceptions
from urllib.request import urlopen
from getpass import getpass
import json
import time

Now we can instantiate the Python Elasticsearch client.

First we prompt the user for their password and Cloud ID.
Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [ ]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# Create the client instance
client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

### Enable Telemetry

Knowing that you are using this notebook helps us decide where to invest our efforts to improve our products. We would like to ask you that you run the following code to let us gather anonymous usage statistics. See [telemetry.py](https://github.com/elastic/elasticsearch-labs/blob/main/telemetry/telemetry.py) for details. Thank you!

In [ ]:
!curl -O -s https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/telemetry/telemetry.py
from telemetry import enable_telemetry

client = enable_telemetry(client, "03-ELSER")

### Test the Client
Before you continue, confirm that the client has connected with this test.

In [ ]:
print(client.info())

Refer to https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect to a self-managed deployment.

Read https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect using API keys.


# Download and Deploy ELSER Model

In this example, we are going to download and deploy the ELSER model in our ML node. Make sure you have an ML node in order to run the ELSER model.

In [ ]:
# delete model if already downloaded and deployed
try:
    client.ml.delete_trained_model(model_id=".elser_model_2", force=True)
    print("Model deleted successfully, We will proceed with creating one")
except exceptions.NotFoundError:
    print("Model doesn't exist, but We will proceed with creating one")

# Creates the ELSER model configuration. Automatically downloads the model if it doesn't exist.
client.ml.put_trained_model(
    model_id=".elser_model_2", input={"field_names": ["text_field"]}
)

The above command will download the ELSER model. This will take a few minutes to complete. Use the following command to check the status of the model download.

In [ ]:
while True:
    status = client.ml.get_trained_models(
        model_id=".elser_model_2", include="definition_status"
    )

    if status["trained_model_configs"][0]["fully_defined"]:
        print("ELSER Model is downloaded and ready to be deployed.")
        break
    else:
        print("ELSER Model is downloaded but not ready to be deployed.")
    time.sleep(5)

Once the model is downloaded, we can deploy the model in our ML node. Use the following command to deploy the model.

In [ ]:
# Start trained model deployment if not already deployed
client.ml.start_trained_model_deployment(
    model_id=".elser_model_2", number_of_allocations=1, wait_for="starting"
)

while True:
    status = client.ml.get_trained_models_stats(
        model_id=".elser_model_2",
    )
    if status["trained_model_stats"][0]["deployment_stats"]["state"] == "started":
        print("ELSER Model has been successfully deployed.")
        break
    else:
        print("ELSER Model is currently being deployed.")
    time.sleep(5)

This also will take a few minutes to complete.

# Indexing Documents with ELSER

In order to use ELSER on our Elastic Cloud deployment we'll need to create an ingest pipeline that contains an inference processor that runs the ELSER model.
Let's add that pipeline using the [`put_pipeline`](https://www.elastic.co/guide/en/elasticsearch/reference/master/put-pipeline-api.html) method.

In [ ]:
client.ingest.put_pipeline(
    id="elser-ingest-pipeline",
    description="Ingest pipeline for ELSER",
    processors=[
        {
            "inference": {
                "model_id": ".elser_model_2",
                "input_output": [
                    {"input_field": "plot", "output_field": "plot_embedding"}
                ],
            }
        }
    ],
)

Let's note a few important parameters from that API call:

- `inference`: A processor that performs inference using a machine learning model.
- `model_id`: Specifies the ID of the machine learning model to be used. In this example, the model ID is set to `.elser_model_2`.
- `input_output`: Specifies input and output fields
- `input_field`: Field name from which the `sparse_vector` representation are created.
- `output_field`:  Field name which contains inference results.

## Create index

To use the ELSER model at index time, we'll need to create an index mapping that supports a [`text_expansion`](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-text-expansion-query.html) query.
The mapping includes a field of type [`sparse_vector`](https://www.elastic.co/guide/en/elasticsearch/reference/master/sparse-vector.html)  to work with our feature vectors of interest.
This field contains the token-weight pairs the ELSER model created based on the input text.

Let's create an index named `elser-example-movies` with the mappings we need.


In [ ]:
client.indices.delete(index="elser-example-movies", ignore_unavailable=True)
client.indices.create(
    index="elser-example-movies",
    settings={"index": {"default_pipeline": "elser-ingest-pipeline"}},
    mappings={
        "properties": {
            "plot": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "plot_embedding": {"type": "sparse_vector"},
        }
    },
)

## Insert Documents
Let's insert our example dataset of 12 movies.

If you get an error, check the model has been deployed and is available in the ML node. In newer versions of Elastic Cloud, ML node is autoscaled and the ML node may not be ready yet. Wait for a few minutes and try again.

In [ ]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/movies.json"
response = urlopen(url)

# Load the response data into a JSON object
data_json = json.loads(response.read())

# Prepare the documents to be indexed
documents = []
for doc in data_json:
    documents.append(
        {
            "_index": "elser-example-movies",
            "_source": doc,
        }
    )

# Use helpers.bulk to index
helpers.bulk(client, documents)

print("Done indexing documents into `elser-example-movies` index!")
time.sleep(3)

Inspect a new document to confirm that it now has an `plot_embedding` field that contains a list of new, additional terms.
These terms are the **text expansion** of the field(s) you targeted for ELSER inference in `input_field` while creating the pipeline.
ELSER essentially creates a tree of expanded terms to improve the semantic searchability of your documents.
We'll be able to search these documents using a `text_expansion` query.

But first let's start with a simple keyword search, to see how ELSER delivers semantically relevant results out of the box.

# Searching Documents

Let's test out semantic search using ELSER.

In [ ]:
response = client.search(
    index="elser-example-movies",
    size=3,
    query={
        "text_expansion": {
            "plot_embedding": {
                "model_id": ".elser_model_2",
                "model_text": "fighting movie",
            }
        }
    },
)

for hit in response["hits"]["hits"]:
    doc_id = hit["_id"]
    score = hit["_score"]
    title = hit["_source"]["title"]
    plot = hit["_source"]["plot"]
    print(f"Score: {score}\nTitle: {title}\nPlot: {plot}\n")

## Next Steps
Now that we have a working example of semantic search using ELSER, you can try it out on your own data. Don't forget to scale down the ML node when you are done.